In [ ]:
"""
Population Model 2024 
Designed for use in preparing annual population forecasts
@author: Kojo Laryea
"""
#1.IMPORT MODULES.
import re
import os
import datetime
import xlsxwriter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests


#2.DEFINE FORECAST WINDOW.
start_period=2025
end_period=2030


#3.DEFINE GEOGRAPHY AND RELATIVE PATH.
Region=['AB','CER','CMA','COC']
Assumptions=['_Fert','_Mort','_Birth','_Net_Mig']
Relative_path=['./Population_Data.xlsx','Population_Cohort.xlsx','Population_Assumptions.xlsx','Population_Exogenous.xlsx']


#4.IMPORT SPREADSHEETS.
##Import Historical Population Data By Geography.The Spreadsheets Also Include The Historical Data For Explanatory Variables.
for i in range(0,4):
    globals()[Region[i] +"_Pop"]=pd.read_excel (os.path.join(os.path.dirname(__file__),'Data',Relative_path[0]),sheet_name=Region[i]).set_index('Date')
   
##Import Single Age Cohorts Population Data By Geography.
for i in range(0,4):
  globals()[Region[i] +"_Cohort"]=pd.read_excel (os.path.join(os.path.dirname(__file__),'Data',Relative_path[1]),sheet_name=Region[i]).set_index('Index')

##Import Fertility, Mortality, Birth, Net Migration And YTD assumptions By Geography.
for i in range(0,4):
    for j in range (0,4):
        globals()[Region[i] +Assumptions[j]]=pd.read_excel (os.path.join(os.path.dirname(__file__),'Data',Relative_path[2]),\
                                                            sheet_name=Region[i]+Assumptions[j]).set_index('Index')

##Import Exogenous Assumptions For Forecasting Purposes.
AB_YTD=pd.read_excel(os.path.join(os.path.dirname(__file__),'Data',Relative_path[2]), sheet_name='AB_YTD').set_index('Index')
Exog=pd.read_excel(os.path.join(os.path.dirname(__file__),'Data',Relative_path[3]), sheet_name='Exogenous').set_index('Date')


#5.GENERATE FORMULAS FOR MOVING AVERAGES,SUMPRODUCT, RESULTS SHEETS AND FORECASTING.
##1.Moving Averages.
def moving_average(x, y):
    return np.convolve(x, np.ones(y), "valid") / y

##2.Sum Product.
def sumproduct(x,y):
    df=np.zeros(len(x))
    if (len(x)==len(y)):
        for i in range(0,len(x)):
            df[i]=x[i]*y[i]
        x=df.sum(axis=0)
    else:
        print ("vector lengths do not match!")
    return x

##3.Results Sheet.
def results_sheet(x,y,z,c):
    df=pd.DataFrame(index=np.arange(253),columns=np.arange(y-x+6)).rename(index={0:'Pop_Tot',1:'Pop_Growth',2:'',3:'Tot_Net_Mig',
         4:'Int_Net_Mig',5:'Dom_Net_Mig',6:'',7:'Births',8:'Deaths',9:'Nat_Inc',10:''})
    df.columns=[str(x) for x in range(x-5, y+1)]
    for i in range(0,242):
        df=df.rename(index={i+11:(c.index[i])})
    for i in range(x-5,x):
        ###Using Numerical Index Due To Cohort Indices, Which Do Not Have A Useable Pattern.
        df[str(i)].iloc[11:253]=round(c[(i)].iloc[0:242],0) 
        df[str(i)]['Pop_Tot']=z.Tot_Pop[i]
        df[str(i)]['Pop_Growth']=round(z.Pop_Growth[i],1)
        df[str(i)]['Tot_Net_Mig']=z.Dom_Net_Mig[i]+z.Int_Net_Mig[i]
        df[str(i)]['Int_Net_Mig']=z.Int_Net_Mig[i]
        df[str(i)]['Dom_Net_Mig']=z.Dom_Net_Mig[i]
        df[str(i)]['Births']=z.Births[i]
        df[str(i)]['Deaths']=z.Deaths[i]
        df[str(i)]['Nat_Inc']=z.Births[i]-z.Deaths[i]
    return df

##4.Forecast Function
def pop_forecast(geog,sheet,int_coef,dom_coef,start_period,end_period):
    for i in range (start_period,end_period+1):
       ###Forecast International and Domestic Net Migration.
       if geog=='AB':
           sheet[str(i)]['Int_Net_Mig']=round(sumproduct(int_coef,[1,(sheet[str(i-1)][0]/Exog['CAN_Tot_Pop'][i-1]),
                                        (Exog['AB_Tot_Emp'][i-1]-Exog['AB_Tot_Emp'][i-2]),(Exog['Imm_Pol'][i])]),0) 
           sheet[str(i)]['Dom_Net_Mig']=round(sumproduct(dom_coef,[1,sheet[str(i-1)]['Dom_Net_Mig'],(Exog['AB_Tot_UR'][i-1]
                                         /Exog['CAN_Tot_UR'][i-1]),0,(Exog['AB_Tot_Emp'][i-1]-Exog['AB_Tot_Emp'][i-2])]),0) 
       elif geog=='CER':
            sheet[str(i)]['Int_Net_Mig']=round(sumproduct(int_coef,[1,(AB[str(i)]['Int_Net_Mig']),(Exog['CER_Tot_UR'][i-1]/Exog['AB_Tot_UR'][i-1])]),0) #type: ignore
            sheet[str(i)]['Dom_Net_Mig']=round(sumproduct(dom_coef,[1,((AB[str(i)]['Dom_Net_Mig'])),(Exog['CER_Tot_UR'][i-1]/Exog['AB_Tot_UR'][i-1])]),0) #type: ignore
        
       elif geog=='CMA':
           sheet[str(i)]['Int_Net_Mig']=round(sumproduct(int_coef,[1, (CER[str(i)]['Int_Net_Mig'])]),0) #type: ignore
           sheet[str(i)]['Dom_Net_Mig']=round(sumproduct(dom_coef,[1, (CER[str(i)]['Dom_Net_Mig'])]),0) #type: ignore
       
       elif geog=='COC':
           sheet[str(i)]['Int_Net_Mig']=(sumproduct(int_coef,[1,(Exog['CER_Tot_UR'][i-1]/Exog['CAN_Tot_UR'][i-1]), (sheet[str(i-1)][0]/Exog['CAN_Tot_Pop'][i-1]),(Exog
                                        ['CER_Tot_Emp'][i-1]-Exog['CER_Tot_Emp'][i-2]),(Exog['COC_Avg_Hprice'][i-1]),(Exog['CMA_CPI'][i-1]),(Exog['Imm_Pol'][i-1])]))
           sheet[str(i)]['Dom_Net_Mig']=0
        
       else:
           print('geography listed cannot be found!')
       
       #Apply Alberta YTD Assumptions To First Forecast Period
       if i==start_period and geog=='AB':    
        sheet[str(start_period)]['Int_Net_Mig']=round(((1-(len(AB_YTD)*0.25))*sheet[str(start_period)]['Int_Net_Mig'])+sum(AB_YTD.Int_Net_Mig),0)
        sheet[str(start_period)]['Dom_Net_Mig']=round(((1-(len(AB_YTD)*0.25))*sheet[str(start_period)]['Dom_Net_Mig'])+sum(AB_YTD.Dom_Net_Mig),0)
       
       sheet[str(i)]['Tot_Net_Mig']=sheet[str(i)]['Int_Net_Mig']+sheet[str(i)]['Dom_Net_Mig'] 
       
       ###Shift Previous Age Cohorts A Year Ahead.
       sheet[str(i)][12:132]= sheet[str(i-1)][11:131]
       sheet[str(i)][133:253]=sheet[str(i-1)][132:252]

       ###Forecast Total Births By Gender as the Age Zero Cohort.
       sheet[str(i)]['m0']=round(globals()[geog + "_Birth"]['Ratio']['Male']*(sumproduct(np.array(globals()[geog + "_Fert"][i]),(sheet[str(i-1)][24:60]))),0)
       sheet[str(i)]['f0']=round(globals()[geog + "_Birth"]['Ratio']['Female']*(sumproduct(np.array(globals()[geog + "_Fert"][i]),(sheet[str(i-1)][24:60]))),0)
       sheet[str(i)]['Births']=sheet[str(i)]['m0']+sheet[str(i)]['f0']
      
       ###Forecast Total Deaths.
       sheet[str(i)]['Deaths']=round(sumproduct(sheet[str(i)][11:253],globals()[geog + "_Mort"][i]),0)

       ###Calculate Natural Increase, Total Population and Population Growth For The Forecast Window.
       sheet[str(i)]['Nat_Inc']=round(sheet[str(i)]['Births']-sheet[str(i)]['Deaths'],0)
       sheet[str(i)]['Pop_Tot']=sheet[str(i-1)]['Pop_Tot']+sheet[str(i)]['Births']-sheet[str(i)]['Deaths']+sheet[str(i)]['Tot_Net_Mig']
       sheet[str(i)]['Pop_Growth']=round(((sheet[str(i)]['Pop_Tot']-sheet[str(i-1)]['Pop_Tot'])/sheet[str(i-1)]['Pop_Tot'])*100,1)

       ###Break Down  Deaths And Net Migration By Cohorts and Subtract and Add Respectively From Age Cohorts
       sheet[str(i)][11:253]=(sheet[str(i)][11:253])-(globals()[geog + "_Mort"][i]*sheet[str(i)][11:253])+\
       (globals()[geog + "_Net_Mig"]['Int']*sheet[str(i)]['Int_Net_Mig'])+(globals()[geog + "_Net_Mig"]['Dom']*sheet[str(i)]['Dom_Net_Mig'])
       sheet[str(i)][11:253]=round(sheet[str(i)][11:253].astype(float),0)

#Generate Results Sheet For AB, CER, CMA,COC.
for i in range(0,4):
    globals()[Region[i]]=results_sheet(start_period,end_period, globals()[Region[i] +"_Pop"], globals()[Region[i] +"_Cohort"])


#6.DEFINE VARIABLES TO BE USED IN INTERNATIONAL AND DOMESTIC NET MIGRATION ESTIMATION BY GEOGRAPHY.
Imm=AB_Pop.Imm_Pol # type: ignore
CER_INM=CER_Pop.Int_Net_Mig #type: ignore
CER_DNM=CER_Pop.Dom_Net_Mig #type: ignore
AB_Res_Dev=AB_Pop.AB_Res_Dev #type: ignore
CMA_Inf=COC_Pop.CMA_CPI.shift(1) #type: ignore
COC_Hprice=COC_Pop.Avg_House_Price.shift(1) #type: ignore
L_AB_Dom_Net_Mig=AB_Pop.Dom_Net_Mig.shift(1) #type: ignore
DL_AB_Tot_Emp=AB_Pop.AB_Tot_Emp.diff().shift(1) #type: ignore
L_4yr_MA_AB_Dom_Net_Mig=AB_Pop.Mov_Avg_AB_Dom_Net_Mig.shift(1) #type: ignore
L_CER_AB_UR_Share=CER_Pop.Tot_UR.shift(1)/AB_Pop.AB_Tot_UR.shift(1) #type: ignore
DL_CER_Tot_Emp=CER_Pop.Tot_Emp.drop(range(1986,1987)).diff().shift(1) #type: ignore
L_CER_CAN_UR_Share=CER_Pop.Tot_UR.shift(1)/AB_Pop.CAN_Tot_UR.shift(1) #type: ignore
L_AB_Can_Pop_Share=AB_Pop.Tot_Pop.shift(1)/AB_Pop.CAN_Tot_Pop.shift(1) #type: ignore
L_AB_Can_UR_Share=AB_Pop.AB_Tot_UR.shift(1)/AB_Pop.CAN_Tot_UR.shift(1) #type: ignore
L_COC_CAN_Pop_Share=COC_Pop.Tot_Pop.shift(1)/AB_Pop.CAN_Tot_Pop.shift(1) #type: ignore


#7.ESTIMATION OF INTERNATIONAL AND DOMESTIC NET MIGRATION BY GEOGRAPHY.
##Create Explanatory And Dependent Variable Blocks.
###Alberta Block.
AB_INM_x=sm.add_constant(pd.concat([L_AB_Can_Pop_Share.drop(range(1976,1978)),DL_AB_Tot_Emp.drop(range(1976,1978)),Imm.drop(range(1976,1978))], axis=1))
AB_DNM_x=sm.add_constant(pd.concat([L_AB_Dom_Net_Mig.drop(range(1976,1980)), L_AB_Can_UR_Share.drop(range(1976,1980)),
                                    AB_Res_Dev.drop(range(1976,1980)),DL_AB_Tot_Emp.drop(range(1976,1980))], axis=1))

AB_INM_y=AB_Pop.Int_Net_Mig.drop(range(1976,1978)) # type: ignore
AB_DNM_y=AB_Pop.Dom_Net_Mig.drop(range(1976,1980)) # type: ignore

###CER Block.
CER_INM_x=sm.add_constant(pd.concat([AB_Pop.Int_Net_Mig.drop(range(1976,2003)),L_CER_AB_UR_Share.drop(range(1976,2003))], axis=1)) # type: ignore
CER_DNM_x=sm.add_constant(pd.concat([AB_Pop.Dom_Net_Mig.drop(range(1976,1988)),L_CER_AB_UR_Share.drop(range(1976,1988))], axis=1)) # type: ignore

CER_INM_y=CER_Pop.Int_Net_Mig.drop(range(1986,2003)) # type: ignore
CER_DNM_y=CER_Pop.Dom_Net_Mig.drop(range(1986,1988)) # type: ignore

###CMA Block.
CMA_INM_x=sm.add_constant(CER_INM.drop(range(1986,2002))) # type: ignore
CMA_DNM_x=sm.add_constant(CER_DNM.drop(range(1986,2002))) # type: ignore

CMA_INM_y=CMA_Pop.Int_Net_Mig.drop(range(1996,2002)) # type: ignore
CMA_DNM_y=CMA_Pop.Dom_Net_Mig.drop(range(1996,2002)) # type: ignore

###COC Block.
COC_TNM_x=sm.add_constant(pd.concat([L_CER_CAN_UR_Share.drop(range(1976,1991)),L_COC_CAN_Pop_Share.drop(range(1961,1991)), DL_CER_Tot_Emp.drop(range(1987,1991)),\
                         COC_Hprice.drop(range(1961,1991)),CMA_Inf.drop(range(1961,1991)), Imm.drop(range(1976,1991))], axis=1)) # type: ignore
COC_TNM_y=COC_Pop.Tot_Net_Mig.drop(range(1961,1991)) # type: ignore


#8.RUN REGRESSION FOR INTERNATIONAL AND DOMESTIC NET MIGRATION BY GEOGRAPHY.
##Alberta Estimation.
AB_INM_estimate=sm.OLS(np.array(AB_INM_y,dtype=float),np.array(AB_INM_x,dtype=float)).fit()
AB_DNM_estimate=sm.OLS(np.array(AB_DNM_y,dtype=float),np.array(AB_DNM_x,dtype=float)).fit()

##CER Estimation.
CER_INM_estimate=sm.OLS(np.array(CER_INM_y,dtype=float),np.array(CER_INM_x,dtype=float)).fit()
CER_DNM_estimate=sm.OLS(np.array(CER_DNM_y,dtype=float),np.array(CER_DNM_x,dtype=float)).fit()

##CMA Estimation.
CMA_INM_estimate=sm.OLS(np.array(CMA_INM_y,dtype=float),np.array(CMA_INM_x,dtype=float)).fit()
CMA_DNM_estimate=sm.OLS(np.array(CMA_DNM_y,dtype=float ),np.array(CMA_DNM_x,dtype=float)).fit()

##COC Estimation.
COC_TNM_estimate=sm.OLS(np.array(COC_TNM_y,dtype=float),np.array(COC_TNM_x,dtype=float)).fit()


#.9 FORECAST POPULATION BY REGION.
pop_forecast('AB',AB,AB_INM_estimate.params,AB_DNM_estimate.params,start_period,end_period)     #type: ignore
pop_forecast('CER',CER,CER_INM_estimate.params,CER_DNM_estimate.params,start_period,end_period) #type: ignore
pop_forecast('CMA',CMA,CMA_INM_estimate.params,CMA_DNM_estimate.params,start_period,end_period) #type: ignore
pop_forecast('COC',COC,COC_TNM_estimate.params,COC_TNM_estimate.params,start_period,end_period) #type: ignore

##Drop Int and Dom Net Migration Rows, which were just created to make the forumla work across regions.
COC.drop(['Dom_Net_Mig'],inplace=True) #type: ignore
COC.drop(['Int_Net_Mig'],inplace=True) #type: ignore


#10.EXPORT POPULATION FORECASTS BY REGION.
export_path='Population_Forecasts_'+str(datetime.datetime.now()).split('.')[0].replace(':','.')+'.xlsx'
with pd.ExcelWriter(export_path) as writer:
   for i in range (0,4):
       globals()[Region[i]].to_excel(writer, sheet_name=Region[i]+'_Pop')

print('Forecasts Complete.See The Output File '+export_path)
